In [195]:
import pandas as pd
import json

# squad

In [197]:
with open('../examples/temp/datasets/predictions_.json', "r", encoding='utf-8') as reader:
    input_pred = json.load(reader)

In [258]:
with open('../examples/temp/datasets/squad/train-v2.0.json', "r", encoding='utf-8') as reader:
    input_real = json.load(reader)['data']

In [263]:
# for i in input_real:
#     for j in i['paragraphs']:
#         for k in j['qas']:
#             if k['is_impossible'] == True:
#                 print(k)
    

In [260]:
len(input_real)

442

In [223]:
temp = {}
for i in input_real[0]['paragraphs']:
    for j in i['qas']:
        temp[j['id']] = [(j['question'], [k['text'] for k in j['answers']][0] if len(j['answers']) > 0 else '')]

In [224]:
for i in input_pred:
    if i in temp:
        temp[i].append(input_pred[i])
    else:
        pass

In [225]:
temp

{'56ddde6b9a695914005b9628': [('In what country is Normandy located?',
   'France'),
  'They were descended from Norse ("Norman" comes from "Norseman")'],
 '56ddde6b9a695914005b9629': [('When were the Normans in Normandy?',
   '10th and 11th centuries'),
  'Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy,'],
 '56ddde6b9a695914005b962a': [('From which countries did the Norse originate?',
   'Denmark, Iceland and Norway'),
  'raiders and pirates from Denmark, Iceland and Norway'],
 '56ddde6b9a695914005b962b': [('Who was the Norse leader?', 'Rollo'),
  'raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King'],
 '56ddde6b9a695914005b962c': [('What century did the Normans first gain their separate identity?',
   '10th century'),
  ''],
 '5ad39d53604f3c001a3fe8d1': [("Who gave their name to Normandy in the 1000's and 1100's",
   ''),
  'The Normans (Norman: Nourmands; French: Normands;

# fever

In [155]:
import pickle

In [176]:
from tqdm import tqdm

In [280]:
df = pd.read_hdf('../examples/temp/datasets/fever/shared_task_dev-new.h5')

In [281]:
# df = pickle.load(open('../examples/temp/datasets/entry.pickle', 'rb'))

In [282]:
entry_ = {}
entry_['title'] = 'fever'
entry_['paragraphs'] = []
some_texts = []
c = 0
for item in df.to_dict('records'):
    for ind in range(len(item['related_line_nums'])):
        if item['label'] != 2:
            qas = [{}]
            qas[0]['question'] = item['claim']
            qas[0]['id'] = 'fever_' + str(c)
            c += 1
            qas[0]['answers'] = [
                {'text': item['related_sentences'][ind][item['related_line_nums'][ind]], 
                 'answer_start': 0 if item['related_line_nums'][ind] == 0 else sum(
                     [len(item['related_sentences'][ind][i]) for i in range(item['related_line_nums'][ind])]
                 )}
            ]
            if item['label'] == 0: # False, wose than impossible
                qas[0]['is_impossible'] = 2
            elif item['label'] == 1: # True, not impossible
                qas[0]['is_impossible'] = 0
            else:
                raise Exception('label error')
            pa = {'qas': qas, 'context': ''.join(item['related_sentences'][ind])}
            some_texts.append(''.join(item['related_sentences'][ind]))
            entry_['paragraphs'].append(pa)
        else:
            if len(some_texts):
                qas = [{}]
                qas[0]['question'] = item['claim']
                qas[0]['id'] = 'fever_' + str(c)
                c += 1
                qas[0]['plausible_answers'] = [],
                qas[0]['answers'] = [],
                if item['label'] == 2: # Impossible
                    qas[0]['is_impossible'] = 1
                else:
                    raise Exception('label error')
                pa = {'qas': qas, 'context': np.random.choice(some_texts)}
                entry_['paragraphs'].append(pa)

In [288]:
entry_['paragraphs'][1]['qas']

[{'question': 'Telemundo is a English-language television network .',
  'id': 'fever_1',
  'answers': [{'text': 'Telemundo -LRB- -LSB- teleˈmundo -RSB- -RRB- is an American Spanish-language terrestrial television network owned by Comcast through the NBCUniversal division NBCUniversal Telemundo Enterprises .',
    'answer_start': 0}],
  'is_impossible': 2}]

# leaders

In [289]:
df = pd.read_pickle('../examples/temp/datasets/leaders/leaders.pickle')

In [290]:
df

,claim,claimant,date,id,label,related_articles,texts
0,A line from George Orwell's novel 1984 predict...,,2017-07-17,0,0,"[122094, 122580, 130685, 134765]",[1984: George Orwell predicted 2017 almost 70 ...
1,Maine legislature candidate Leslie Gibson insu...,,2018-03-17,1,2,"[106868, 127320, 128060]",[Republican who criticized Parkland students d...
2,A 17-year-old girl named Alyssa Carson is bein...,,2018-07-18,4,1,"[132130, 132132, 149722]",[FIRST PERSON TO COMPLETE NASA’S PASSPORT PROG...
3,In 1988 author Roald Dahl penned an open lette...,,2019-02-04,5,2,"[123254, 123418, 127464]",[How dangerous is measles? Tweet\n\n27 March 2...
4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,2016-03-22,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]",[Remarks on counterterrorism at Stanford Unive...
5,"Rhode Island is ""almost dead last"" among North...",Leonidas Raptakis,2014-02-11,7,2,"[8284, 3768, 20091, 82368, 73148, 4493]",[LIS > Code of Virginia > 18.2-10\nprev | next...
6,The poorest counties in the U.S. are in Appala...,Jim Webb,2014-11-19,8,1,"[70709, 70708]","[Counties in Appalachia\n\nAlabama: Bibb, Blou..."
7,Koch Industries paid the legal fees of George ...,,2013-07-18,9,0,"[120591, 120592, 127866, 129483]",[UPDATE Confronting False Allegations about th...
8,"""Minnesota, Michigan, Iowa already have 70 mph...",Robin Vos,2013-08-22,11,1,"[69547, 80095, 7994, 81116, 77621]","[Robin Vos Discusses Milwaukee Crime, Speed Li..."
9,"""FBI Uniform Crime Report for 2016 shows more ...",Nick Schroer,2017-10-17,12,1,"[72012, 26005, 43481, 55671]",[FBI: Over Four Times More People Stabbed to D...


In [275]:
entry_ = {}
entry_['title'] = 'leaders'
entry_['paragraphs'] = []
some_texts = []
c = 0
for item in df.to_dict('records'):

    qas = [{}]
    qas[0]['question'] = item['claim']
    qas[0]['id'] = 'leaders_' + str(c)
    c += 1
    qas[0]['answers'] = [
        {'text': 'unknown', 
         'answer_start': 0}
    ]
    qas[0]['is_impossible'] = item['label']
   
    pa = {'qas': qas, 'context': ''.join(item['texts'])}

    entry_['paragraphs'].append(pa)

In [279]:
entry_['paragraphs'][0]

{'qas': [{'question': "A line from George Orwell's novel 1984 predicts the power of smartphones.",
   'id': 'leaders_0',
   'answers': [{'text': 'unknown', 'answer_start': 0}],
   'is_impossible': 0}],
 'context': '1984: George Orwell predicted 2017 almost 70 years ago\nApril, 1984. 13:00. Comrade 6079, Winston Smith, thinks a thought, starts a diary, and falls in love. But Big Brother is always watching. Set in a world where an invasive government keeps a malevolently watchful eye on its citizens, this radical and much-lauded staging explores surveillance, identity and why Orwell?s vision of the future is as relevant now as ever. Now with a brand new Australian cast, we bring adapter-directors Robert Icke and Duncan Macmillan?s thrilling vision back to Australia for a national tour.\n\nTHERE was no crystal ball, no religious apparitions or voices from the dead when George Orwell wrote Nineteen Eighty-Four, accurately depicted our present almost 70 years ago.\n\nIt’s no coincidence tha

# fnc

In [227]:
ts = pd.read_csv('../examples/temp/datasets/fnc/train_stances.csv')

tb = pd.read_csv('../examples/temp/datasets/fnc/train_bodies.csv')

bodyid2text = {}

for i in tb.to_dict('records'):
    bodyid2text[i['Body ID']] = i['articleBody']

stance2lab = {'agree': 3, 'disagree': 0, 'discuss': 2, 'unrelated': 1}

entry_ = {}
entry_['title'] = 'fnc'
entry_['paragraphs'] = []
some_texts = []
c = 0
for item in ts.to_dict('records'):
    
    qas = [{}]
    qas[0]['question'] = item['Headline']
    qas[0]['id'] = 'fnc_' + str(c)
    c += 1
    qas[0]['answers'] = [
        {'text': 'unknown', 
         'answer_start': 0}
    ]
    qas[0]['is_impossible'] = stance2lab[item['Stance']]
    
    pa = {'qas': qas, 'context': bodyid2text[item['Body ID']]}
    
    entry_['paragraphs'].append(pa)


In [17]:
import collections

In [18]:
collections.Counter(ts.Stance.tolist())

Counter({'unrelated': 36545, 'agree': 3678, 'disagree': 840, 'discuss': 8909})

In [13]:
ts.loc[0].tolist()

["Police find mass graves with at least '15 bodies' near Mexico town where 43 students disappeared after police clash",
 712,
 'unrelated']